# Chicago Crashes: Phase 3 Project

### Jesse Markowitz working notebook

## Stakeholders & Main Problem

Our stakeholder will be the City of Chicago. We'll focus on identifying the factors that most contribute to an increasing severity of crashes (fatalities, serious injury). 

We'll look closely at the columns associated with fatalities and injuries to create targets for this multi-class problem.

## Import datasets & poke around

Datasets located at:
 - https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3if
 - https://data.cityofchicago.org/Transportation/Traffic-Crashes-Vehicles/68nd-jvt3
 - https://data.cityofchicago.org/Transportation/Traffic-Crashes-People/u6pd-qa9d

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
crashes_df = pd.read_csv('./data/Traffic_Crashes_-_Crashes.csv')
people_df = pd.read_csv('./data/Traffic_Crashes_-_People.csv')
vehicles_df = pd.read_csv('./data/Traffic_Crashes_-_Vehicles.csv')

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (12,13,20,21,22,23,24,25,26,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
crashes_df.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,JC343143,NaN,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,JA329216,NaN,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,JD292400,NaN,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,3.0,0.0,10,6,7,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008)
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,JD293602,NaN,07/11/2020 01:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,...,0.0,0.0,3.0,0.0,1,7,7,41.802119,-87.622115,POINT (-87.622114914961 41.802118543011)
4,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...,JD290451,NaN,07/08/2020 02:00:00 PM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,14,4,7,NaN,NaN,NaN


In [4]:
# From Angie: create a df to show % of missing values from another df:

# percent_missing = df.isnull().sum() * 100 / len(df)
# missing_value_df = pd.DataFrame({'column_name': df.columns,
#                                  'percent_missing': percent_missing})

# pd.set_option('display.max_rows', None)
# missing_value_df

In [5]:
crashes_df.isna().mean()

CRASH_RECORD_ID                  0.000000
RD_NO                            0.008486
CRASH_DATE_EST_I                 0.924181
CRASH_DATE                       0.000000
POSTED_SPEED_LIMIT               0.000000
TRAFFIC_CONTROL_DEVICE           0.000000
DEVICE_CONDITION                 0.000000
WEATHER_CONDITION                0.000000
LIGHTING_CONDITION               0.000000
FIRST_CRASH_TYPE                 0.000000
TRAFFICWAY_TYPE                  0.000000
LANE_CNT                         0.632318
ALIGNMENT                        0.000000
ROADWAY_SURFACE_COND             0.000000
ROAD_DEFECT                      0.000000
REPORT_TYPE                      0.024997
CRASH_TYPE                       0.000000
INTERSECTION_RELATED_I           0.773254
NOT_RIGHT_OF_WAY_I               0.952689
HIT_AND_RUN_I                    0.698892
DAMAGE                           0.000000
DATE_POLICE_NOTIFIED             0.000000
PRIM_CONTRIBUTORY_CAUSE          0.000000
SEC_CONTRIBUTORY_CAUSE           0

In [6]:
crashes_df.loc[crashes_df.PRIM_CONTRIBUTORY_CAUSE == 'UNABLE TO DETERMINE'].NUM_UNITS.value_counts()

2.0     178933
1.0      11661
3.0       9474
4.0       1856
5.0        446
6.0        102
7.0         40
8.0         10
9.0          9
14.0         2
11.0         1
12.0         1
Name: NUM_UNITS, dtype: int64

In [7]:
people_df.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [8]:
people_df.isna().mean()

PERSON_ID                0.000000
PERSON_TYPE              0.000000
CRASH_RECORD_ID          0.000000
RD_NO                    0.008475
VEHICLE_ID               0.019737
CRASH_DATE               0.000000
SEAT_NO                  0.795515
CITY                     0.264286
STATE                    0.255537
ZIPCODE                  0.328293
SEX                      0.015122
AGE                      0.287098
DRIVERS_LICENSE_STATE    0.409957
DRIVERS_LICENSE_CLASS    0.491509
SAFETY_EQUIPMENT         0.002918
AIRBAG_DEPLOYED          0.018905
EJECTION                 0.012161
INJURY_CLASSIFICATION    0.000496
HOSPITAL                 0.820056
EMS_AGENCY               0.886213
EMS_RUN_NO               0.981366
DRIVER_ACTION            0.206386
DRIVER_VISION            0.206662
PHYSICAL_CONDITION       0.205814
PEDPEDAL_ACTION          0.981416
PEDPEDAL_VISIBILITY      0.981456
PEDPEDAL_LOCATION        0.981417
BAC_RESULT               0.205471
BAC_RESULT VALUE         0.998735
CELL_PHONE_USE

In [9]:
people_df.INJURY_CLASSIFICATION.value_counts()

NO INDICATION OF INJURY     1096283
NONINCAPACITATING INJURY      55296
REPORTED, NOT EVIDENT         32092
INCAPACITATING INJURY         10827
FATAL                           656
Name: INJURY_CLASSIFICATION, dtype: int64

In [10]:
vehicles_df.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,829999,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,JD124535,01/22/2020 06:25:00 AM,1,DRIVER,NaN,796949.0,NaN,INFINITI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2,PARKED,NaN,834819.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,04/13/2020 10:50:00 PM,2,DRIVER,NaN,827212.0,NaN,BUICK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
vehicles_df.isna().mean()

KeyboardInterrupt: 

## EDA

In [ ]:
crashes_df.columns

In [ ]:
sns.pairplot(crashes_df[['POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'WEATHER_CONDITION', 
                         'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'ROADWAY_SURFACE_COND',
                         'ROAD_DEFECT', 'DAMAGE', 'BEAT_OF_OCCURRENCE', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 
                         'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
                         'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
                         'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH']])

## Dropping columns/basic cleaning

####  First with `.isna().mean()`>80%

In [22]:
def dropna_columns(df, threshold=.8):
    '''
    Drop columns from a Pandas DataFrame in which more than a certain
    percentage (default=80%) of their rows are null/NaN.
    '''
    return df.dropna(thresh=(((1-threshold) * df.shape[0])) + 1, axis=1)

In [23]:
crashes_dropped_df = dropna_columns(crashes_df)
vehicles_dropped_df = dropna_columns(vehicles_df)
people_dropped_df = dropna_columns(people_df)

In [24]:
crashes_dropped_df.shape

(541142, 41)

In [25]:
vehicles_dropped_df.shape

(1107267, 19)

In [26]:
people_dropped_df.shape

(1195747, 22)

In [27]:
vehicles_dropped_df.columns

Index(['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE', 'UNIT_NO',
       'UNIT_TYPE', 'VEHICLE_ID', 'MAKE', 'MODEL', 'LIC_PLATE_STATE',
       'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE',
       'TRAVEL_DIRECTION', 'MANEUVER', 'OCCUPANT_CNT', 'AREA_01_I',
       'FIRST_CONTACT_POINT'],
      dtype='object')

In [28]:
people_dropped_df.columns

Index(['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'RD_NO', 'VEHICLE_ID',
       'CRASH_DATE', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE',
       'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT',
       'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'DRIVER_ACTION',
       'DRIVER_VISION', 'PHYSICAL_CONDITION', 'BAC_RESULT'],
      dtype='object')

In [31]:
people_dropped_df.SEAT_NO.value_counts()

3.0     118468
6.0      41772
4.0      32990
5.0      11585
1.0      10461
2.0       9784
12.0      6193
7.0       5581
10.0      5323
11.0      2081
8.0        274
Name: SEAT_NO, dtype: int64

####  Next by content we don't want/need

In [17]:
people_dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195747 entries, 0 to 1195746
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1195747 non-null  object 
 1   PERSON_TYPE            1195747 non-null  object 
 2   CRASH_RECORD_ID        1195747 non-null  object 
 3   RD_NO                  1185613 non-null  object 
 4   VEHICLE_ID             1172146 non-null  float64
 5   CRASH_DATE             1195747 non-null  object 
 6   SEAT_NO                244512 non-null   float64
 7   CITY                   879728 non-null   object 
 8   STATE                  890189 non-null   object 
 9   ZIPCODE                803192 non-null   object 
 10  SEX                    1177665 non-null  object 
 11  AGE                    852450 non-null   float64
 12  DRIVERS_LICENSE_STATE  705542 non-null   object 
 13  DRIVERS_LICENSE_CLASS  608027 non-null   object 
 14  SAFETY_EQUIPMENT  

**Columns to drop:**

Crashes:
 - `RD_NO` is a police ID number, withheld for recent crashes; reduntant w/`CRASH_RECORD_ID`
 - `REPORT_TYPE` not relevant to conditions in a crash
 - `DATE_POLICE_NOTIFIED`
 - `STREET_NO`, `STREET_DIRECTION`, `STREET_NAME` are all location related; just use lat/long

Vehicles:
 - `RD_NO` is a police ID number, withheld for recent crashes; reduntant w/`CRASH_RECORD_ID`
 - `CRASH_DATE` (redundant w/`crashes_df`)
 - `UNIT_NO` is redundant (count of num vehicles in crash)
 - `AREA_01_I` is ??? Totally unclear what this is
 
People:
 - `RD_NO` is a police ID number, withheld for recent crashes; reduntant w/`CRASH_RECORD_ID`
 - `CRASH_DATE` (redundant w/`crashes_df`)
 - `CITY` and `ZIPCODE`; just use `STATE` to determine driver residence

In [18]:
crashes_cols_to_drop = ['RD_NO', 'REPORT_TYPE', 'DATE_POLICE_NOTIFIED', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME']
vehicles_cols_to_drop = ['RD_NO', 'CRASH_DATE', 'UNIT_NO', 'AREA_01_I']
people_cols_to_drop = ['RD_NO', 'CRASH_DATE', 'CITY', 'ZIPCODE']

In [19]:
crashes_dropped_df = crashes_dropped_df.drop(crashes_cols_to_drop, axis=1)
vehicles_dropped_df = vehicles_dropped_df.drop(vehicles_cols_to_drop, axis=1)
people_dropped_df = people_dropped_df.drop(people_cols_to_drop, axis=1)

In [20]:
crashes_dropped_df.shape

(541142, 35)

In [21]:
vehicles_dropped_df.shape

(1107267, 15)

In [22]:
people_dropped_df.shape

(1195747, 18)

## First Simple Model

### Create a `TARGET` columns

To create our FSM, we'll modify the `MOST_SEVERE_INJURY` column to create a `TARGET` column:
 - 0 = `NO INDICATION OF INJURY`, `NONINCAPACITATING INJURY`, `REPORTED, NOT EVIDENT`
 - 1 = `INCAPACITATING INJURY`, `FATAL`

In [23]:
crashes_dropped_df.MOST_SEVERE_INJURY.value_counts()

NO INDICATION OF INJURY     468994
NONINCAPACITATING INJURY     39675
REPORTED, NOT EVIDENT        21763
INCAPACITATING INJURY         9040
FATAL                          547
Name: MOST_SEVERE_INJURY, dtype: int64

In [24]:
crashes_dropped_df['TARGET'] = crashes_dropped_df['MOST_SEVERE_INJURY'].map({'NO INDICATION OF INJURY': 0,
                                                                              'NONINCAPACITATING INJURY': 0,
                                                                              'REPORTED, NOT EVIDENT': 0,
                                                                              'INCAPACITATING INJURY': 1,
                                                                              'FATAL': 1})

In [25]:
crashes_dropped_df.TARGET.value_counts()

0.0    530432
1.0      9587
Name: TARGET, dtype: int64

In [26]:
crashes_dropped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 36 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   CRASH_DATE                     541142 non-null  object 
 2   POSTED_SPEED_LIMIT             541142 non-null  int64  
 3   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 4   DEVICE_CONDITION               541142 non-null  object 
 5   WEATHER_CONDITION              541142 non-null  object 
 6   LIGHTING_CONDITION             541142 non-null  object 
 7   FIRST_CRASH_TYPE               541142 non-null  object 
 8   TRAFFICWAY_TYPE                541142 non-null  object 
 9   LANE_CNT                       198968 non-null  float64
 10  ALIGNMENT                      541142 non-null  object 
 11  ROADWAY_SURFACE_COND           541142 non-null  object 
 12  ROAD_DEFECT                   

### Apply a dummy model 

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

In [44]:
cols_to_drop = ['TARGET', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL', 
                'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 
                'INJURIES_UNKNOWN', 'CRASH_RECORD_ID', 'CRASH_DATE', 'LATITUDE', 'LONGITUDE', 'LOCATION']

X = crashes_dropped_df.drop(cols_to_drop, axis=1)
y = crashes_dropped_df['TARGET']

y.fillna(value=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [45]:
fsm = DummyClassifier(strategy='most_frequent', random_state=42)
fsm.fit(X_train, y_train)
y_train_preds = fsm.predict(X_train)
y_test_preds = fsm.predict(X_test)

In [46]:
print(classification_report(y_train, y_train_preds))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### Merging all the data into one dataframe

To avoid too many repeated rows from `crashes_dropped_df`, we'll use the `CRASH_RECORD_ID` to only merge in rows from `people_dropped_df` where `DRIVER_TYPE == DRIVER`, then use the `VEHICLE_ID` to only merge in that driver's vehicle from `vehicles_dropped_df`.

This will allow us to still retain over 75% of the rows from `people`.

In [ ]:
people_dropped_df.PERSON_TYPE.value_counts(normalize=True)